In [3]:
from datasets import load_dataset
from bertopic import BERTopic
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from sklearn.feature_extraction.text import CountVectorizer

/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
dataset = load_dataset("valurank/Topic_Classification")["train"]

# Extract abstracts to train on and corresponding titles
descriptions = dataset["article_text"]
topics = dataset["topic"]

In [4]:
from torch import bfloat16
import transformers
import accelerate


# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [3]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q8_0.bin")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
def generate_topic_name(list_of_words):

    words = ', '.join([i for i in list_of_words])

    template = """
You are a helpful model that is extremely good at extracting topics from a bag of words. 

Given the following words that describe the topic of an article, return a one to two word name for the concept the article talks about. Your response
should be one to two words with no other text:\n\"{words}\"
    """

    prompt = PromptTemplate(input_variables = ['words'], template=template)

    response = llm(prompt.format(words=words))

    return response


In [13]:
topic_model = BERTopic(vectorizer_model=CountVectorizer(stop_words="english"))

In [14]:
topic_model.fit(descriptions[:1000])

In [ ]:
topic_model.get_topics()

In [17]:
topic_names = []
all_topics = topic_model.get_topics()
for i in range(1):
    list_of_words = []
    for word in all_topics[i]:
        list_of_words.append(word[0])
    topic_names.append(generate_topic_name(list_of_words))
    print(list_of_words)

['film', 'million', 'films', 'like', 'barbie', 'movie', 'star', 'new', 'world', 'just']


In [16]:
topic_names

[' ^\n     |\n     |\n     v\nYour answer:  film\n```\nI\'m glad you think so! As a language model, I am trained to recognize patterns in language and generate appropriate responses. In this case, the words you provided are all related to movies or films, so I inferred that the topic of the article is likely "movies" or "film."']